In [ ]:
# %% [markdown]
# # Codificação de Dados Agrícolas — Práticas e Percepções de Degradação
# Este notebook padroniza e codifica as colunas dos dados já explodidos
# para permitir análises estatísticas posteriores.

# %%
import pandas as pd
import numpy as np

# %% [markdown]
# ## 1. Carregar os dados
# Ajuste o caminho e o nome do arquivo conforme o dataset que estiver usando.
# Exemplo: 04_Uso_Convencionais_Degradacao_Mun.csv ou 04_Uso_Sustentavel_Degradacao_Mun.csv

# %%
file_path = r"C:/Users/Valentine/Artigo_Mapitos/data_clean/05_Regionais/Regionais/05_Uso_Convencionais_Degradacao_Mun.csv"
df = pd.read_csv(file_path)

print("Dimensões do arquivo:", df.shape)
df.head()

# %% [markdown]
# ## 2. Padronizar nomes das colunas
# Garante que as colunas estejam no mesmo formato esperado.

# %%
df.columns = df.columns.str.strip().str.lower()

# Detectar automaticamente as colunas relevantes
col_mun = [c for c in df.columns if "municipio" in c][0]
col_degrad = [c for c in df.columns if "degradacao" in c][0]
col_pratica = [c for c in df.columns if "pratica" in c][0]

print("Colunas detectadas:")
print("Município:", col_mun)
print("Degradação:", col_degrad)
print("Prática:", col_pratica)

# %% [markdown]
# ## 3. Codificar o nível de degradação como variável ordinal

import unidecode

# Limpeza de texto antes do mapeamento
df[col_degrad] = (
    df[col_degrad]
    .astype(str)
    .str.strip()
    .str.lower()
    .apply(unidecode.unidecode)  # remove acentos
)

# Dicionário ajustado sem acentos
mapa_degradacao = {
    "nao possui degradacao": 0,
    "pouco degradado": 1,
    "degradacao media": 2,
    "muito degradado": 3
}

# Aplicar mapeamento
df["degradacao_ord"] = df[col_degrad].map(mapa_degradacao)

# Conferir valores únicos e se deu certo
print("Valores únicos de degradação bruta:", df[col_degrad].unique())
print("Valores únicos de degradação codificada:", df["degradacao_ord"].unique())


# %% [markdown]
# ## 4. Criar colunas dummies para as práticas
# Transforma cada prática em uma variável booleana (0/1) indicando presença.

# %%
# Normalizar nomes de práticas
df[col_pratica] = df[col_pratica].str.strip().str.lower()

# Criar uma coluna por prática
praticas_dummies = pd.get_dummies(df[col_pratica], prefix="pratica")

# Combinar no dataframe
df = pd.concat([df, praticas_dummies], axis=1)

print("Número de práticas codificadas:", len(praticas_dummies.columns))
df.head()

# %% [markdown]
# ## 5. Calcular o número total de práticas por linha e por município

# %%
# Número de práticas ativas por linha (já é 1 se uma linha = uma prática)
df["n_praticas_linha"] = df.filter(like="pratica_").sum(axis=1)

# Número total de práticas distintas por município
df_mun = df.groupby(col_mun)[df.filter(like="pratica_").columns].any().reset_index()
df_mun["n_praticas_total"] = df_mun[df.filter(like="pratica_").columns].sum(axis=1)

print("Amostra agregada por município:")
df_mun.head()

# %% [markdown]
# ## 6. Exportar arquivos codificados
# - `df_codificado`: base linha a linha (cada prática observada)
# - `df_mun`: base agregada por município

# %%
output_path_linha = r"C:/Users/Valentine/Artigo_Mapitos/data_clean/07_Codificado/079_Convencionais_Degradacao_Linha.csv"
output_path_mun = r"C:/Users/Valentine/Artigo_Mapitos/data_clean/07_Codificado/079_Convencionais_Degradacao_Municipio.csv"

import os
os.makedirs(os.path.dirname(output_path_linha), exist_ok=True)

df.to_csv(output_path_linha, index=False, encoding="utf-8-sig")
df_mun.to_csv(output_path_mun, index=False, encoding="utf-8-sig")

print("✅ Arquivos exportados com sucesso:")
print(output_path_linha)
print(output_path_mun)


Dimensões do arquivo: (975, 5)
Colunas detectadas:
Município: municipio_ibge
Degradação: qual_o_nivel_de_degradacao_que_predomina_nas_pastagens_
Prática: uso_de_praticas_convencionais
Valores únicos de degradação bruta: ['nao possui degradacao' 'pouco degradado' 'degradacao media'
 'muito degradado']
Valores únicos de degradação codificada: [0 1 2 3]
Número de práticas codificadas: 10
Amostra agregada por município:
✅ Arquivos exportados com sucesso:
C:/Users/Valentine/Artigo_Mapitos/data_clean/07_Codificado/079_Convencionais_Degradacao_Linha.csv
C:/Users/Valentine/Artigo_Mapitos/data_clean/07_Codificado/079_Convencionais_Degradacao_Municipio.csv
